### Attendance Streak

In [88]:
import pandas as pd
import pyodbc

In [89]:
server = '157.119.230.120,4071'    
database = 'PowerBI_LISV2'      
username = 'sa'
password = 'Y0m@Sql.!.123'

# Connection string
conn_str = f"""
    DRIVER={{ODBC Driver 17 for SQL Server}};
    SERVER={server};
    DATABASE={database};
    UID={username};
    PWD={password};
    TrustServerCertificate=yes;
"""

In [90]:
x = ['AmulManagement', 'BeamSuntoryManagement', 'GCPLManagement','JohnsonJohnsonManagement','ParleManagement','ColgateManagement']

# Static query parts
designation_list = ['Merchandiser', 'Promoter', 'BA', 'BP', 'ISR', 'FloaterPromoter', 'Floater Promoter']

In [91]:
designation_str = ",".join(f"'{d}'" for d in designation_list)

In [92]:
conn = pyodbc.connect(conn_str)
Attendance_df = pd.DataFrame()

In [93]:
for value in x:
    query2 = f"""
    WITH Attendance_CTE AS (
        SELECT
            '{value}' AS client,
            M.EmpPositionCode,
            M.EmpPositionCodeStatus,
            A.UserName,
            A.FullName,
            A.Region,
            A.Designation,
            A.Status,
            A.[Date] AS Date2,
            M.Status AS chek,
            M.LastWorkingDate,
            CASE
                WHEN A.Status = 'Present' THEN 'P'
                WHEN A.Status = 'Absent' THEN 'A'
                WHEN A.Status = 'Leave' THEN 'L'
                WHEN A.Status = 'Compensatory Off' THEN 'CO'
                WHEN A.Status IN ('Week Off', 'WeekOff') THEN 'WO'
                WHEN A.Status = 'Meeting' THEN 'M'
                WHEN A.Status = 'Training' THEN 'T'
                WHEN A.Status IN ('National Hoilday', 'Holiday') THEN 'H'
                ELSE 'NA'
            END AS AttStatus,
            FORMAT(TRY_CONVERT(date, A.[Date], 105), 'dd') AS DayNum
        FROM PowerBI_LISV2.[{value}].ISPAttendanceMasterExport AS A
        LEFT JOIN (
            SELECT K.*
            FROM (
                SELECT *,
                       RANK() OVER (PARTITION BY UserName ORDER BY UserMasterId DESC) AS RNK
                FROM PowerBI_LISV2.[{value}].UserMaster
            ) AS K
            WHERE K.RNK = 1
        ) AS M ON A.UserName = M.UserName
        WHERE MONTH(TRY_CONVERT(date, A.[Date], 105)) = 10
          AND A.Month = 10 AND Year =2025
          AND (A.UserName LIKE '%ISP%' OR A.UserName LIKE '%MR%')
    )
    SELECT *,
           RANK() OVER (PARTITION BY UserName ORDER BY Date2 DESC) AS CURR_DAYFLAG
    FROM Attendance_CTE;
    
    
    """
    df = pd.read_sql(query2, conn)
    Attendance_df = pd.concat([Attendance_df, df], ignore_index=True)
    print(f"Running for: {value}")

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_28272\1141041286.py:48: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query2, conn)


Running for: AmulManagement
Running for: BeamSuntoryManagement
Running for: GCPLManagement
Running for: JohnsonJohnsonManagement
Running for: ParleManagement
Running for: ColgateManagement


In [94]:
conn = pyodbc.connect(conn_str)
# Attendance_df = pd.read_sql(query2, conn)

In [95]:
Attendance_df=Attendance_df.drop_duplicates()

In [96]:

# --- Step 1: Prepare Data ---
df = Attendance_df.copy()
df['Date2'] = pd.to_datetime(df['Date2'], errors='coerce', dayfirst=True)
df = df.sort_values(['UserName', 'Date2'])

# --- Step 2: Normalize to one row per user-date ---
daily = (
    df.assign(AttStatus=df['AttStatus'].str.strip().str.upper())
      .groupby(['UserName', 'Date2'], as_index=False)
      .agg({'AttStatus': 'first'})  # keep one record per date
)

# --- Step 3: Define which statuses break the streak ---
presence_status = ['P', 'WO', 'M']

def calc_absent_streak(user_df):
    user_df = user_df.sort_values('Date2').reset_index(drop=True)
    streak = 0
    streaks = []
    for status in user_df['AttStatus']:
        if status == 'A':
            streak += 1
        elif status in presence_status:
            streak = 0  # break the streak
        else:
            streak = 0  # treat all other statuses as breaking it too
        streaks.append(streak)
    user_df['AbsentStreak'] = streaks
    return user_df

# --- Step 4: Apply per user ---
streak_df = daily.groupby('UserName', group_keys=False).apply(calc_absent_streak).reset_index(drop=True)

# --- Step 5: Merge back with original dataset (no duplicates) ---
final_df = df.merge(streak_df[['UserName', 'Date2', 'AbsentStreak']], on=['UserName', 'Date2'], how='left')

# --- Step 6: Add AttendanceDate (for readability) ---
# final_df['AttendanceDate'] = final_df['Date2'].dt.strftime('%d-%m-%Y')

# --- Step 7: Output ---
# final_output = final_df[['UserName', 'FullName', 'Region', 'Designation', 'AttendanceDate', 'AttStatus', 'AbsentStreak']]

# print(final_output.tail(15))


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_28272\2822677721.py:32: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  streak_df = daily.groupby('UserName', group_keys=False).apply(calc_absent_streak).reset_index(drop=True)


In [97]:
final_df=final_df.drop_duplicates()

In [98]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32151 entries, 0 to 32150
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   client                 32151 non-null  object        
 1   EmpPositionCode        19298 non-null  object        
 2   EmpPositionCodeStatus  19298 non-null  object        
 3   UserName               32151 non-null  object        
 4   FullName               32151 non-null  object        
 5   Region                 32151 non-null  object        
 6   Designation            32151 non-null  object        
 7   Status                 32017 non-null  object        
 8   Date2                  32145 non-null  datetime64[ns]
 9   chek                   32151 non-null  object        
 10  LastWorkingDate        1016 non-null   object        
 11  AttStatus              32151 non-null  object        
 12  DayNum                 32151 non-null  object        
 13  C

In [99]:
final_df.head(1)

client EmpPositionCode EmpPositionCodeStatus  UserName  \
0  AmulManagement            None                  None  AMLMR027   

                   FullName Region   Designation   Status      Date2  chek  \
0  Tejaswini Sachin Shedage   West  Merchandiser  Present 2025-10-01  True   

  LastWorkingDate AttStatus DayNum  CURR_DAYFLAG  AbsentStreak  
0            None         P     01            13           0.0

In [100]:
latest_streak = (
    final_df.sort_values(['UserName', 'Date2'])
      .groupby('UserName')['AbsentStreak']
      .last()
      .reset_index()
      .rename(columns={'AbsentStreak': 'recent_absent_streak'})
)

In [101]:
latest_streak[latest_streak['UserName']=='CLG-ISP-1122']

UserName  recent_absent_streak
432  CLG-ISP-1122                   3.0

In [102]:
final_df = final_df.merge(latest_streak, on='UserName', how='left')

In [103]:
final_df[final_df['UserName']=='CLG-ISP-1122']

client  EmpPositionCode EmpPositionCodeStatus      UserName  \
5240  ColgateManagement  LISF-CLG-KA-479                Active  CLG-ISP-1122   
5241  ColgateManagement  LISF-CLG-KA-479                Active  CLG-ISP-1122   
5242  ColgateManagement  LISF-CLG-KA-479                Active  CLG-ISP-1122   
5243  ColgateManagement  LISF-CLG-KA-479                Active  CLG-ISP-1122   
5244  ColgateManagement  LISF-CLG-KA-479                Active  CLG-ISP-1122   
5245  ColgateManagement  LISF-CLG-KA-479                Active  CLG-ISP-1122   
5246  ColgateManagement  LISF-CLG-KA-479                Active  CLG-ISP-1122   
5247  ColgateManagement  LISF-CLG-KA-479                Active  CLG-ISP-1122   
5248  ColgateManagement  LISF-CLG-KA-479                Active  CLG-ISP-1122   
5249  ColgateManagement  LISF-CLG-KA-479                Active  CLG-ISP-1122   

            FullName Region   Designation            Status      Date2  chek  \
5240  Kishankumar HR  South  Merchandiser           Present 2025-10-01  True   
5241  Kishankumar HR  South  Merchandiser           Present 2025-10-02  True   
5242  Kishankumar HR  South  Merchandiser           Present 2025-10-03  True   
5243  Kishankumar HR  South  Merchandiser           Present 2025-10-04  True   
5244  Kishankumar HR  South  Merchandiser           Present 2025-10-05  True   
5245  Kishankumar HR  South  Merchandiser  Compensatory Off 2025-10-06  True   
5246  Kishankumar HR  South  Merchandiser           Present 2025-10-07  True   
5247  Kishankumar HR  South  Merchandiser            Absent 2025-10-08  True   
5248  Kishankumar HR  South  Merchandiser            Absent 2025-10-09  True   
5249  Kishankumar HR  South  Merchandiser            Absent 2025-10-13  True   

     LastWorkingDate AttStatus DayNum  CURR_DAYFLAG  AbsentStreak  \
5240            None         P     01            10           0.0   
5241            None         P     02             9           0.0   
5242            None         P     03             8           0.0   
5243            None         P     04             7           0.0   
5244            None         P     05             6           0.0   
5245            None        CO     06             5           0.0   
5246            None         P     07             4           0.0   
5247            None         A     08             3           1.0   
5248            None         A     09             2           2.0   
5249            None         A     13             1           3.0   

      recent_absent_streak  
5240                   3.0  
5241                   3.0  
5242                   3.0  
5243                   3.0  
5244                   3.0  
5245                   3.0  
5246                   3.0  
5247                   3.0  
5248                   3.0  
5249                   3.0

In [104]:
final_df.to_csv("Attendence_STREAK2.csv")